## Used care listings
We are working with dataset fof used cars from eBay 'Kleinanzeigen', a classfieds section of the German eBay website. The dataset was originally scraped and uploaded to Kaggle, now with a few modifications from the original dataset:

+ now sampled 50,000 data points from the full dataset to ensure the code runs quickly in hosted enviroment
+ Dirtied the dataset a bit to more closely resemable what would have been expected from a scraped dataset

The aim of this proejct is to clearn the data and analyze the included used car listings.


In [1]:
import pandas as pd

dateparser = lambda x:pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
autos = pd.read_csv('autos.csv',encoding='Latin-1',parse_dates=['dateCrawled','dateCreated','lastSeen'],date_parser=dateparser)
autos.head()



,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null datetime64[ns]
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null datetime64[ns]
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50

## Observations
+ data volume of 50000
+ memory usage of 7.6+ MB
+ 20 columns of mostly strings
+ some columns have null values, not more than 20% of the data
+ The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
autos = autos.rename(columns={'dateCrawled':'date_crawled', 
                              'yearOfRegistration':'registration_year',
                      'monthOfRegistration':'registration_month',
                      'notRepairedDamage':'unrepaired_damage',
                      'dateCreated':'ad_created',
                      'lastSeen':'last_seen',
                      'postalCode':'postal_code',
                     'nrOfPictures':'number_pictures',
                     'fuelType':'fuel_type',
                     'powerPS':'power_ps',
                     'vehicleType':'vehicle_type',
                     'offerType':'offer_type'})

In [5]:
autos.head()


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to e more descriptive

In [6]:
autos.unrepaired_damage



0        nein
1        nein
2        nein
3        nein
4        nein
5         NaN
6         NaN
7        nein
8        nein
9        nein
10       nein
11        NaN
12       nein
13       nein
14        NaN
15       nein
16       nein
17       nein
18        NaN
19       nein
20        NaN
21       nein
22       nein
23       nein
24       nein
25         ja
26       nein
27        NaN
28         ja
29       nein
         ... 
49970    nein
49971    nein
49972      ja
49973    nein
49974    nein
49975    nein
49976    nein
49977    nein
49978      ja
49979    nein
49980     NaN
49981    nein
49982     NaN
49983     NaN
49984     NaN
49985     NaN
49986    nein
49987    nein
49988    nein
49989      ja
49990    nein
49991     NaN
49992    nein
49993     NaN
49994    nein
49995    nein
49996    nein
49997    nein
49998    nein
49999    nein
Name: unrepaired_damage, Length: 50000, dtype: object

may be there is a need to thoroughly look through the entry representations of the difference between 'nein' and NaN entries

In [7]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-12 16:06:22,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
first,2016-03-05 14:06:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-06-11 00:00:00,NaN,NaN,2016-03-05 14:45:46
last,2016-04-07 14:36:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-07 00:00:00,NaN,NaN,2016-04-07 14:58:50
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN


## Observations
+ let's investigate more into power_ps
+ seems registration year and month might be stored as strings and may need some cleaning
+ We may also have to clean up the trailing longs into floats in each column depending on our usage


+ There are a number of text columns where all (or nearly all) of the values are the same:
+ + seller
+ +  offer_type
+ The num_photos column looks odd, we'll need to investigate this further.

In [8]:
autos = autos.drop(['seller','offer_type','number_pictures'],axis=1)

In [9]:
# import re

# autos.loc[:,['price','odometer']].replace(to_replace='\$([0-9,\.]+).*', value=r'\1',regex=True,inplace=True)


In [10]:
autos['price'] = [i.replace('$','').replace(',','') if '$' in i else i for i in autos['price']]
autos['price'] = autos['price'].astype('int')

In [11]:
autos['odometer'] = [i.replace('km','').replace(',','') if 'km' in i else i for i in autos['odometer']]
autos['odometer'] = autos['price'].astype('int')

In [12]:
autos[['price','odometer']].dtypes

price       int64
odometer    int64
dtype: object

In [13]:
autos = autos.rename(columns={'odometer':'odometer_km'})

In [14]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,5000,3,lpg,peugeot,nein,2016-03-26,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,8500,6,benzin,bmw,nein,2016-04-04,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,8990,7,benzin,volkswagen,nein,2016-03-26,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,4350,6,benzin,smart,nein,2016-03-12,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,1350,7,benzin,ford,nein,2016-04-01,39218,2016-04-01 14:38:50


went ahead and took out some unnecessary characters to more easily manipulate data

In [15]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head(20)

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

In [16]:
autos.odometer_km.value_counts()

0         1421
500        781
1500       734
2500       643
1000       639
1200       639
600        531
800        498
3500       498
2000       460
999        434
750        433
900        420
650        419
850        410
700        395
4500       394
300        384
2200       382
950        379
1100       376
1300       371
3000       365
550        356
1800       355
5500       340
1250       335
350        335
1600       327
1999       322
          ... 
46200        1
29600        1
13480        1
21700        1
7373         1
3279         1
4286         1
188          1
17830        1
9130         1
910          1
238          1
2671         1
69900        1
151990       1
2479         1
4510         1
86500        1
47499        1
16998        1
27299        1
41850        1
4780         1
686          1
6495         1
20790        1
8970         1
846          1
2895         1
33980        1
Name: odometer_km, Length: 2357, dtype: int64

In [17]:
autos.price.value_counts()

0         1421
500        781
1500       734
2500       643
1000       639
1200       639
600        531
800        498
3500       498
2000       460
999        434
750        433
900        420
650        419
850        410
700        395
4500       394
300        384
2200       382
950        379
1100       376
1300       371
3000       365
550        356
1800       355
5500       340
1250       335
350        335
1600       327
1999       322
          ... 
46200        1
29600        1
13480        1
21700        1
7373         1
3279         1
4286         1
188          1
17830        1
9130         1
910          1
238          1
2671         1
69900        1
151990       1
2479         1
4510         1
86500        1
47499        1
16998        1
27299        1
41850        1
4780         1
686          1
6495         1
20790        1
8970         1
846          1
2895         1
33980        1
Name: price, Length: 2357, dtype: int64

In [18]:
autos[['price','odometer_km']].sort_values(by=['price','odometer_km'],ascending=False)

,price,odometer_km
39705,99999999,99999999
42221,27322222,27322222
27371,12345678,12345678
39377,12345678,12345678
47598,12345678,12345678
2897,11111111,11111111
24384,11111111,11111111
11137,10000000,10000000
47634,3890000,3890000
7814,1300000,1300000


In [19]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [20]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

There are a number of listings with prices below \$30, including about 1,500 at \$0. There are also a small number of listings with very high values, including 14 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. We will keep the \$1 items, but remove anything above \$350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [21]:
# autos = autos[(autos[['price','odometer_km']] > 0).all(axis=1) & (autos[['price','odometer_km']] < 169000).all(axis=1)]

In [22]:
autos = autos[autos['price'].between(1,351000)]
autos.price.describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

# cleaning summary
+ did a sort values of both columns
+ found repeating dubious integer values, went ahead to index in to smallest range out of those values 


## Exploring the date columns
There are a number of columns with date information:

+ date_crawled
+ registration_month
+ registration_year
+ ad_created
+ last_seen
These are a combination of dates that were crawled, and dates with meta-information from the crawler. The non-registration dates are stored as strings.

We'll explore each of these columns to learn more about the listings.

In [23]:
autos = autos.sort_values(by=['date_crawled','ad_created','last_seen'],ascending=True)


sorted autos by date_crawled, ad_created and last_seen from ascending order 

In [24]:
autos['date_crawled'] = pd.to_datetime(autos['date_crawled']).dt.date

In [25]:
autos['last_seen'] = pd.to_datetime(autos['last_seen']).dt.date

changed last_seen and date_crawled columns to datetime date objects instead of time stamp for percentage distrubutions 

In [26]:
autos['date_crawled'].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [27]:
autos['ad_created'].value_counts(normalize=True,dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [28]:
autos['last_seen'].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [29]:
import datetime as dt
march_date_crawled = autos.loc[(autos['date_crawled'] >= dt.date(2016,3,1)) & (autos['date_crawled'] <= dt.date(2016,4,1)),'date_crawled'].value_counts(normalize=True, dropna=False)

In [30]:
march_date_crawled.sort_index()

2016-03-05    0.029053
2016-03-06    0.016109
2016-03-07    0.041311
2016-03-08    0.038194
2016-03-09    0.037957
2016-03-10    0.036918
2016-03-11    0.037367
2016-03-12    0.042351
2016-03-13    0.017975
2016-03-14    0.041926
2016-03-15    0.039327
2016-03-16    0.033966
2016-03-17    0.036280
2016-03-18    0.014810
2016-03-19    0.039894
2016-03-20    0.043461
2016-03-21    0.042870
2016-03-22    0.037839
2016-03-23    0.036965
2016-03-24    0.033659
2016-03-25    0.036257
2016-03-26    0.036942
2016-03-27    0.035666
2016-03-28    0.039989
2016-03-29    0.039115
2016-03-30    0.038642
2016-03-31    0.036517
2016-04-01    0.038642
Name: date_crawled, dtype: float64

sample percentage distribution of march by date_crawled column

In [31]:
march_last_seen = autos.loc[(autos['last_seen'] > dt.date(2016,3,1)) & (autos['last_seen'] < dt.date(2016,4,1)),'last_seen'].value_counts(normalize=True, dropna=False)

In [32]:
march_last_seen.sort_index()

2016-03-05    0.002525
2016-03-06    0.010196
2016-03-07    0.012721
2016-03-08    0.017479
2016-03-09    0.022626
2016-03-10    0.025151
2016-03-11    0.029180
2016-03-12    0.056079
2016-03-13    0.020975
2016-03-14    0.029715
2016-03-15    0.037434
2016-03-16    0.038794
2016-03-17    0.066226
2016-03-18    0.017333
2016-03-19    0.037337
2016-03-20    0.048699
2016-03-21    0.048650
2016-03-22    0.050398
2016-03-23    0.043698
2016-03-24    0.046611
2016-03-25    0.045300
2016-03-26    0.039619
2016-03-27    0.036900
2016-03-28    0.049184
2016-03-29    0.052680
2016-03-30    0.058409
2016-03-31    0.056079
Name: last_seen, dtype: float64

Sample of distribution percentage of march last_seen logs

In [33]:
march_ad_created = autos.loc[(autos['ad_created'] > dt.date(2016,3,1)) & (autos['ad_created'] < dt.date(2016,4,1)),'ad_created'].value_counts(normalize=True,dropna=False)

In [34]:
march_ad_created.sort_index()

2016-03-02    0.000123
2016-03-03    0.001033
2016-03-04    0.001770
2016-03-05    0.027343
2016-03-06    0.018294
2016-03-07    0.041482
2016-03-08    0.039786
2016-03-09    0.039589
2016-03-10    0.038089
2016-03-11    0.039294
2016-03-12    0.043892
2016-03-13    0.020311
2016-03-14    0.042023
2016-03-15    0.040622
2016-03-16    0.035974
2016-03-17    0.037351
2016-03-18    0.016229
2016-03-19    0.040228
2016-03-20    0.045318
2016-03-21    0.044876
2016-03-22    0.039171
2016-03-23    0.038286
2016-03-24    0.034966
2016-03-25    0.037917
2016-03-26    0.038532
2016-03-27    0.037007
2016-03-28    0.041777
2016-03-29    0.040646
2016-03-30    0.040007
2016-03-31    0.038064
Name: ad_created, dtype: float64

sample of percentage distribution of march ad_created logs

## Summary of exploratory date_crawled,  ad_created, and last_seen logs
+ sorted autos by date_crawled, ad_created and last_seen from ascending order
+ changed last_seen and date_crawled columns to datetime date objects instead of time stamp for percentage distrubutions
+ provided some sample percentages distribution logs for each column

In [35]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [36]:
autos.registration_year = pd.DatetimeIndex(autos.registration_year).year

In [38]:
autos.registration_year

29899    1970
26440    1970
20681    1970
12426    1970
9524     1970
25599    1970
47236    1970
2469     1970
12739    1970
21321    1970
4022     1970
4392     1970
2679     1970
10947    1970
20014    1970
14331    1970
24640    1970
8008     1970
9723     1970
44030    1970
40570    1970
16456    1970
9867     1970
41632    1970
29531    1970
21468    1970
49416    1970
30783    1970
34139    1970
24771    1970
         ... 
44551    1970
48576    1970
17151    1970
23409    1970
11538    1970
24929    1970
34041    1970
16304    1970
29938    1970
21527    1970
8036     1970
12119    1970
20677    1970
4719     1970
19282    1970
37268    1970
43691    1970
8576     1970
44797    1970
2213     1970
19809    1970
49569    1970
9363     1970
14150    1970
45786    1970
7982     1970
20941    1970
4752     1970
4378     1970
16947    1970
Name: registration_year, Length: 48565, dtype: int64

In [ ]:
autos['registration_year'] = autos[(autos['registration_year'] > 1900) & (autos['registration_year'] < 2016)]   

In [ ]:
autos.registration_year

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [ ]:
autos.registration_year.value_counts(normalize=True)